In [1]:
# Funny problem
import sys
import os
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import breadth_first_order
import math

#Returns the shortest path from source to the sink
def ShortestPath(arr, s, t):
    source = s
    temp = [t]
    i = t
    while arr[i] != -9999:
        temp.append(arr[i])
        i = arr[i]
    return temp[::-1]


#Returns the augmented path
def augmentedPath(A, shortestpath, bottleneck_edge):
    for i in range(1, len(shortestpath)):
        start, end = shortestpath[i - 1], shortestpath[i]
        A[start][end] -= bottleneck_edge
        A[end][start] += bottleneck_edge
    return A

def bottleneck(A, arr):
    minval = np.inf
    for i in range(1, len(arr)):
        start, end = arr[i - 1], arr[i]
        if A[start][end] < minval:
            minval = A[start][end]
    return minval

#Calculates the maximum flow through the network
def MaxFlow(G, s, t):
    flow = 0
    source, sink = s, t

    nodes, parent = breadth_first_order(csr_matrix(G), 0, directed=True, return_predecessors=True)

    shortestpath = ShortestPath(parent, source, sink)
    dim = math.sqrt(((len(G) - 2))/2)
    path = []
    for i in range(len(shortestpath)):
        val = shortestpath[i]
        if (i == 0):
            continue
        if (i == len(shortestpath) - 1):
            continue
        if (val%2 == 0):
            temp = (int)(val/2)
        else: 
            temp = (int)(val/2) + 1
        if (temp%dim == 0):
            x = (int)(temp/dim)
            y = (int)(dim)
        else:
            x = (int)(temp/dim) + 1       
            y = (int)(temp%dim)
        path.append([x,y])
    pathorig = []
    for indx in range(len(path)):
        if(indx%2 == 0):
            pathorig.append(path[indx])
    
    while source in shortestpath:
        bottleneck_edge = bottleneck(G, shortestpath)
        flow += bottleneck_edge
        G = augmentedPath(G, shortestpath, bottleneck_edge)
        nodes, parent = breadth_first_order(csr_matrix(G), 0, directed=True, return_predecessors=True)
        shortestpath = ShortestPath(parent, source, sink)
        path = []
        for i in range(len(shortestpath)):
            val = shortestpath[i]
            if (i == 0):
                continue
            if (i == len(shortestpath) - 1):
                continue
            if (val%2 == 0):
                temp = (int)(val/2)
            else: 
                temp = (int)(val/2) + 1
            if (temp%dim == 0):
                x = (int)(temp/dim)
                y = (int)(dim)
            else:
                x = (int)(temp/dim) + 1       
                y = (int)(temp%dim)
            path.append([x,y])
            i = i + 1
        path1 = []
        for indx in range(len(path)):
            if(indx%2 == 0):
                path1.append(path[indx])
        pathorig.append(path1)
    return flow,pathorig


#Splits each node to Vin and Vout and connects the starting nodes to source
def Split(n, st):
    # Total number of nodes
    g = n * n * 2 + 2
    x1 = np.zeros((g, g), dtype=np.int)

    # Vin,Vout edges are given a value 1
    for j in range(1, n * n * 2, 2):
        x1[j][j + 1] = 1

    # Connect to Sink
    for j in range(1, n + 1):
        x1[2 * j][g - 1] = 1
        x1[2 * (n - 1) * n + 2 * j][g - 1] = 1

    for j in range(n):
        x1[j * 2 * n + 2][g - 1] = 1
        x1[2 * n * (j + 1)][g - 1] = 1

    # Connect Source to Vin of vertices
    for l in st:
        x1[0][(l[0] - 1) * 2 * n + (2 * (l[1] - 1)) + 1] = 1

    # Connect each vertex to its adjacent vertices
    # Right
    for l in range(2, n * n * 2 + 1, 2):
        if l % (2 * n) != 0:
            x1[l][l + 1] = 1
            # x[l + 2][l - 1] = 1

    # Left
    for l in range(2, n * n * 2 + 1, 2):
        if (l - 2) % (2 * n) != 0:
            x1[l][l - 3] = 1

    # Top
    for l in range(2, n * n * 2 + 1, 2):
        if l - (2 * n) > 0:
            x1[l][l - (2 * n) - 1] = 1

    # Bottom
    for l in range(2, n * n * 2 + 1, 2):
        if l + (2 * n) < n * n * 2 + 1:
            x1[l][l + (2 * n) - 1] = 1
            
    maxflow,a = MaxFlow(x1, 0, g - 1)

    return maxflow,a

if __name__ == '__main__':
    line = []
    file_path =  "/Users/rishithaguptha/Downloads/data.txt"
    with open(file_path) as f:
        for l in f:
            line.append((map(int, l.strip().split())))

    n, tc = line[0]
    flag = 0
    grid_dimension = n
    grid_vertices = []
    
    for k in range(tc):
        if tc > n*n:
            flag = 1
            break
        x, y = line[k + 1]
        if (x>=1 and y>=1 and x<=n and y<=n):
            grid_vertices.append((x, y))
        else:
            flag = 1

    if flag == 0 :
        final_flow,sequence = Split(grid_dimension, grid_vertices)
        if final_flow == len(grid_vertices):
            print("Yes, a solution exists")
            for i in range(len(sequence)):
                if(i==0):
                    print("Path from",sequence[i], ":",sequence[i]) 
                
                elif(len(sequence[i])!=0):
                    print("Path from",sequence[i][0], ":",sequence[i]) 
        else:
            print("Solution doesn't exist")
    else:
        print("Invalid Input")




Solution doesn't exist
